In [8]:

import pandas as pd
import requests
import re
from io import StringIO

url = "https://estadisticas.bcrp.gob.pe/estadisticas/series/api/PD04639PD-PD04640PD-PD04637PD-PD04638PD/csv/2020-01-01/2021-11-30"
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
req = requests.get(url, headers=headers)
req_text=str(req.text).replace("<br>","\n")
req_text=str(req_text).replace('"',"")

dict_months = {
    "Ene": "01",
    "Feb": "02",
    "Mar": "03",
    "Abr": "04",
    "May": "05",
    "Jun": "06",
    "Jul": "07",
    "Ago": "08",
    "Set": "09",
    "Oct": "10",
    "Nov": "11",
    "Dic": "12"
}

pattern = re.compile(r'\b(' + '|'.join(dict_months.keys()) + r')\b')
result = pattern.sub(lambda x: dict_months[x.group()], req_text)
data = StringIO(result)

df=pd.read_csv(data, names=["FECHA","TC_SBS_COMPRA","TC_SBS_VENTA","TC_INTERBANCARIO_COMPRA","TC_INTERBANCARIO_VENTA"],skiprows=1)
df['FECHA']=pd.to_datetime(df['FECHA'], format='%d.%m.%y')
df



,FECHA,TC_SBS_COMPRA,TC_SBS_VENTA,TC_INTERBANCARIO_COMPRA,TC_INTERBANCARIO_VENTA
0,2020-01-02,3.301000,3.303143,3.301,3.305
1,2020-01-03,3.310571,3.312000,3.308,3.311
2,2020-01-06,3.322000,3.324000,3.320,3.323
3,2020-01-07,3.325571,3.326571,3.325,3.327
4,2020-01-08,3.317571,3.319143,3.318,3.320
...,...,...,...,...,...
462,2021-11-02,4.006167,4.008667,4.007,4.012
463,2021-11-03,4.010667,4.012500,4.008,4.014
464,2021-11-04,4.007833,4.010667,4.007,4.013
465,2021-11-05,4.013167,4.014833,4.010,4.017


In [12]:

def round_half_up(number,decimals):
    if round((number * 10 ** (decimals + 1)) % 10 )== 5:
        new_number=number+1/(10**(decimals+1))
        return round(new_number, decimals)
    else:
        return round(number, decimals)

df['TC_SBS_COMPRA']=df['TC_SBS_COMPRA'].apply(round_half_up,decimals=3)
df['TC_SBS_VENTA']=df['TC_SBS_VENTA'].apply(round_half_up,decimals=3)

df.to_csv("../results/dolar.csv",index=False)


In [21]:
df_sbs_com=df[["FECHA","TC_SBS_COMPRA"]]
df_sbs_ven=df[["FECHA","TC_SBS_VENTA"]]
df_int_com=df[["FECHA","TC_INTERBANCARIO_COMPRA"]]
df_int_ven=df[["FECHA","TC_INTERBANCARIO_VENTA"]]

df_sbs_com.columns=['time','value']
df_sbs_ven.columns=['time','value']
df_int_com.columns=['time','value']
df_int_ven.columns=['time','value']

df_sbs_com['time']=df_sbs_com['time'].dt.strftime('%Y-%m-%d')
df_sbs_ven['time']=df_sbs_ven['time'].dt.strftime('%Y-%m-%d')
df_int_com['time']=df_int_com['time'].dt.strftime('%Y-%m-%d')
df_int_ven['time']=df_int_ven['time'].dt.strftime('%Y-%m-%d')


df_sbs_com.to_json("../results/timeseries_by_tc/TC_SBS_COMPRA.json",orient="records")
df_sbs_ven.to_json("../results/timeseries_by_tc/TC_SBS_VENTA.json",orient="records")
df_int_com.to_json("../results/timeseries_by_tc/TC_INTERBANCARIO_COMPRA.json",orient="records")
df_int_ven.to_json("../results/timeseries_by_tc/TC_INTERBANCARIO_VENTA.json",orient="records")


/tmp/ipykernel_13060/2451044916.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sbs_com['time']=df_sbs_com['time'].dt.strftime('%Y-%m-%d')
/tmp/ipykernel_13060/2451044916.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sbs_ven['time']=df_sbs_ven['time'].dt.strftime('%Y-%m-%d')
/tmp/ipykernel_13060/2451044916.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen